In [3]:
!jupyter nbconvert --to html "/content/drive/MyDrive/Colab Notebooks/자연어처리_기말고사대비_감정분석.ipynb"


[NbConvertApp] Converting notebook /content/drive/MyDrive/Colab Notebooks/자연어처리_기말고사대비_감정분석.ipynb to html
[NbConvertApp] Writing 408225 bytes to /content/drive/MyDrive/Colab Notebooks/자연어처리_기말고사대비_감정분석.html


In [2]:
import pandas as pd

save_path = "/content/drive/MyDrive/자연어처리_시험대체/output/utterance_joined_multilabel.csv"

df = pd.read_csv(save_path)

In [3]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 127.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 44.8 MB/s eta 0:00:00


# 언어 패턴 분석 (KoNLPy + 규칙기반 Feature 비율)

In [4]:
FEATURE_PATTERNS = {
    # 1) 시간적 확장 / 미래 일반화
    "tense_time_expansion": [
        r"앞으로도", r"계속", r"영원히", r"평생", r"내일도",
        r"앞으로는", r"이제\s*부터", r"언제나", r"언젠가", r"다시는",
        r"영원토록", r"끝없이", r"평생토록", r"무한히", r"끝까지",
        r"앞으로\s*영원히", r"앞으로\s*계속", r"영원한", r"언제까지나", r"미래에도"
    ],

    # 2) 문장 확정성 / 단정적 표현
    "sentence_certainty": [
        r"거야", r"확실", r"틀림없", r"무조건", r"분명",
        r"당연", r"절대", r"백퍼", r"100%", r"완전",
        r"반드시", r"꼭", r"필히", r"확실히", r"명백",
        r"정해졌", r"확언", r"확답", r"확신", r"틀림없어"
    ],

    # 3) 자기 지시성 (1인칭 집중)
    "self_referential": [
        r"\b나는\b", r"\b내가\b", r"\b내\b", r"\b나\b", r"\b내\s*탓\b",
        r"\b내\s*잘못\b", r"\b내\s*문제\b", r"\b내\s*입장\b", r"\b내\s*인생\b", r"\b내\s*삶\b",
        r"내가\s*왜", r"내가\s*어쩌다", r"내\s*마음", r"나\s*자신", r"나를",
        r"나한테", r"나에게", r"내가\s*한", r"내가\s*또", r"내가\s*이렇게"
    ],

    # 4) 부정성 편향 / 부정 예언
    "negativity_bias": [
        r"못\s*할\s*거야", r"안\s*될\s*거야", r"소용없", r"망했", r"안돼",
        r"안된다", r"글렀다", r"끝났어", r"희망\s*없", r"의미\s*없",
        r"불가능", r"절대\s*못", r"절망적", r"무의미", r"멈춰버리",
        r"해봤자", r"어차피", r"어림없", r"물\s*건너갔", r"끝장"
    ],

    # 5) 과잉일반화·흑백논리 (항상/전부/절대):contentReference[oaicite:2]{index=2}
    "absolute_all_or_nothing": [
        r"항상", r"절대", r"전부", r"모두", r"완전",
        r"한\s*번도", r"단\s*한\s*번도", r"아무것도", r"아무도", r"늘",
        r"도무지", r"모든\s*것", r"온통", r"모조리", r"언제나",
        r"모든\s*사람", r"누구도", r"언제든", r"결코", r"백\s*퍼센트"
    ],

    # 6) 이분법적 평가 척도 / 흑백논리 (완벽‧최악 등):contentReference[oaicite:3]{index=3}
    "dichotomous_scale": [
        r"완벽", r"최고", r"최악", r"끝이야", r"0\s*점",
        r"100\s*점", r"성공\s*아니면\s*실패", r"다\s*망", r"전혀\s*아니", r"흑백",
        r"죽거나\s*살거나", r"도\s*아니면\s*모", r"올인", r"0\s*아니면\s*1", r"백\s*아니면\s*흑",
        r"완벽해야", r"결정적", r"절대적", r"두\s*가지\s*뿐", r"극과\s*극"
    ],

    # 7) 파국화 / 재난화 (재앙적 예측):contentReference[oaicite:4]{index=4}
    "catastrophizing": [
        r"큰일", r"망한다", r"인생\s*끝", r"끝장", r"돌이킬\s*수\s*없",
        r"최악의\s*상황", r"다\s*무너", r"다\s*끝났", r"재앙", r"죽을\s*것",
        r"파멸", r"완전히\s*무너", r"깨져버려", r"폭망", r"폭락",
        r"세상이\s*끝", r"완전히\s*끝", r"대재앙", r"멸망", r"절체절명"
    ],

    # 8) 예언 / 미래예측 (성급한 결론)
    "fortune_telling": [
        r"분명히\s*.*거야", r"뻔해", r"결국", r"또\s*.*거야", r"어차피\s*.*거야",
        r"이번에도\s*.*거야", r"확실히\s*.*될\s*거야", r"당연히\s*.*될\s*거야", r"필연적으로\s*.*될\s*것", r"틀림없이\s*.*될\s*거다",
        r"머지않아\s*.*될\s*거야", r"곧\s*.*될\s*것", r"결국에는\s*.*하고야\s*말\s*것", r"조만간\s*.*할\s*거야", r"이미\s*.*결정됐다",
        r"곧장\s*.*될\s*거야", r"반드시\s*.*하게\s*된다", r"곧\s*.*일어날\s*거야", r"결국\s*.*리라", r"조만간\s*.*일\s*것"
    ],

    # 9) 독심술 / 의도단정:contentReference[oaicite:5]{index=5}
    "mind_reading": [
        r"분명\s*나를", r"쟤는\s*내", r"속으로\s*날", r"내\s*생각엔\s*.*나를", r"날\s*무시",
        r"날\s*싫어", r"나\s*별로래", r"날\s*비웃", r"나를\s*멀리", r"나를\s*피하",
        r"나를\s*좋아하지", r"나를\s*사랑하지", r"분명히\s*.*생각할\s*거야", r"당연히\s*.*생각하고\s*있", r"분명히\s*.*싫어하",
        r"분명히\s*.*좋아하", r"나에\s*대해\s*.*생각하", r"그\s*사람은\s*.*라고\s*생각할\s*거야", r"내가\s*싫어서", r"날\s*미워"
    ],

    # 10) 개인화 / 과도한 자기탓:contentReference[oaicite:6]{index=6}
    "personalization": [
        r"내\s*탓이야", r"내가\s*.*해서", r"나\s*때문에", r"내\s*때문이야", r"내가\s*망쳤",
        r"내가\s*문제야", r"다\s*내\s*잘못했어", r"내가\s*그랬어", r"내가\s*해서\s*그래", r"다\s*내\s*책임",
        r"나로\s*인해서", r"내가\s*일을\s*그렇게\s*해서", r"내가\s*만들어서", r"내가\s*하고\s*말았어", r"내가\s*그걸\s*망쳤어",
        r"내가\s*일을\s*그르쳤어", r"내\s*잘못이야", r"내가\s*끝장냈어", r"내\s*실수였어", r"내가\s*만든\s*결과야"
    ],

    # 11) 당위/의무/규범 강요 (should statements)
    "should_statements": [
        r"해야\s*해", r"해야지", r"반드시", r"무조건\s*해야", r"당연히\s*해야",
        r"그러면\s*안\s*돼", r"하면\s*안\s*돼", r"절대\s*하면\s*안", r"해야만", r"해야\s*한다",
        r"마땅히\s*해야", r"해야\s*할\s*것", r"해야\s*옳다", r"해야만\s*한다", r"반드시\s*해야\s*한다",
        r"당연히\s*해야\s*만\s*한다", r"필히\s*해야", r"그러면\s*안\s*되는\s*거야", r"하지\s*말아야", r"해야\s*되는\s*거야"
    ],

    # 12) 낙인찍기 / 고정된 정체성 (명명 오류):contentReference[oaicite:7]{index=7}
    "labeling_identity": [
        r"나는\s*원래", r"난\s*그냥", r"내가\s*무슨", r"난\s*.*사람", r"무능",
        r"찌질", r"한심", r"별로", r"못난", r"실패자",
        r"쓰레기", r"루저", r"나는\s*한심한\s*인간", r"나는\s*실패자", r"나는\s*쓰레기",
        r"나는\s*못난\s*놈", r"나는\s*무능력자", r"난\s*아무\s*것도\s*아닌\s*존재", r"나는\s*아무\s*가치가\s*없다", r"나는\s*.*이상한\s*사람"
    ],

    # 13) 비교 / 열등감 (자기평가 왜곡)
    "comparison_inferiority": [
        r"나만", r"다른\s*사람은", r"쟤는", r"걔는", r"다들\s*.*하는데",
        r"비교하면", r"나는\s*왜\s*이래", r"나보다", r"보다\s*잘", r"보다\s*많이",
        r"보다\s*예쁘", r"보다\s*잘하는", r"남들은", r"남들처럼", r"남들과\s*비교",
        r"남들이", r"전부\s*다", r"나만\s*빼고", r"다른\s*애들은", r"나만\s*못"
    ],

    # 14) 긍정 무시 / 성과 축소:contentReference[oaicite:8]{index=8}:contentReference[oaicite:9]{index=9}
    "disqualify_positive": [
        r"별거\s*아니", r"그건\s*그냥", r"운이\s*좋", r"우연", r"원래\s*그\s*정도",
        r"대단한\s*거\s*아니", r"당연히\s*된\s*거", r"그냥\s*우연", r"초심자의\s*행운", r"별\s*의미\s*없",
        r"가볍게\s*넘겨", r"하찮은", r"우연히", r"별로\s*대단하지", r"그럴\s*수도\s*있는",
        r"그건\s*그냥\s*.*때문", r"큰\s*일\s*아니", r"원래\s*이런\s*것", r"당연히\s*생긴", r"작은\s*일"
    ],

    # 15) 강조 부사 / 과장 (과장·축소와 관련)
    "intensifiers": [
        r"진짜", r"완전", r"너무", r"엄청", r"되게",
        r"겁나", r"미친듯", r"최상", r"최악", r"완전히",
        r"아주", r"매우", r"정말", r"극도로", r"정말로",
        r"대단히", r"몹시", r"극히", r"도대체", r"무척"
    ],

    # 16) 단정적 인과 / 단일 원인 (원인 단일화)
    "single_cause_blame": [
        r"다\s*.*때문이야", r"전부\s*.*탓", r"그래서\s*무조건", r"결론은", r"원인은\s*딱",
        r"무조건\s*.*때문", r"모든\s*게\s*.*때문", r"모두\s*.*탓", r"이\s*모든\s*것은\s*.*때문", r"100%\s*.*때문",
        r"온전히\s*.*탓", r"다\s*.*탓", r"나쁜\s*것은\s*.*때문", r"다\s*.*잘못", r"이유는\s*.*하나",
        r"다른\s*이유\s*없어", r"오로지\s*.*때문", r"전부\s*.*때문이다", r"모든\s*원인은\s*.*", r"단지\s*.*때문"
    ],

    # 17) 요구 / 기대 / 타인에게 당위 강요
    "demandingness": [
        r"해줘야", r"해야\s*하는데", r"왜\s*안\s*해", r"왜\s*그렇게\s*해", r"당연히\s*해줘야",
        r"그게\s*맞지", r"해야지\s*왜\s*안", r"해야만\s*해", r"얼른\s*해", r"해줘",
        r"해야\s*되는\s*거야", r"해야\s*되는\s*거잖아", r"왜\s*해주지\s*않", r"해라", r"해야\s*한다고\s*생각해",
        r"해줘야\s*하는\s*게", r"해줄\s*필요가", r"그러면\s*안\s*돼", r"하면\s*안\s*돼", r"하지\s*말아야\s*돼"
    ],
}


In [5]:
# ===============================
# 1️⃣ 언어 패턴 분석
# ===============================

import re
import pandas as pd
from konlpy.tag import Okt

okt = Okt()

# 1. 분석 텍스트 컬럼 확정
df["analysis_text"] = (
    df["model_input_text"]
    .fillna("")
    .where(df["model_input_text"].fillna("").str.len() > 0, df["text"].fillna(""))
)

# 2. 언어 Feature 규칙 정의 (어절/표현은 추후 확장 가능)


# 3. Feature 추출 함수
def extract_language_features(text: str):
    text = text or ""

    eojeol = text.split()
    total_eojeol = max(len(eojeol), 1)

    morphs = okt.morphs(text, stem=True)
    total_morphs = max(len(morphs), 1)

    result = {
        "total_eojeol": len(eojeol),
        "total_morphs": len(morphs)
    }

    for feat, patterns in FEATURE_PATTERNS.items():
        count = sum(len(re.findall(p, text)) for p in patterns)
        result[f"{feat}_count"] = count
        result[f"{feat}_ratio_eojeol"] = count / total_eojeol
        result[f"{feat}_ratio_morphs"] = count / total_morphs

    return result

# 4. DF에 Feature 컬럼 추가
lang_feat_df = df["analysis_text"].apply(extract_language_features).apply(pd.Series)
df = pd.concat([df, lang_feat_df], axis=1)

df.head()


,utterance_id,sentence_labels,sentence_scores,model_input_text,utterance_labels,utterance_multihot,utterance_label_names,id,age,gender,...,disqualify_positive_ratio_morphs,intensifiers_count,intensifiers_ratio_eojeol,intensifiers_ratio_morphs,single_cause_blame_count,single_cause_blame_ratio_eojeol,single_cause_blame_ratio_morphs,demandingness_count,demandingness_ratio_eojeol,demandingness_ratio_morphs
0,0,[10],[0.9981536269187927],안녕하세요.,[10],[0 0 0 0 0 0 0 0 0 0 1],['비인지왜곡'],D075,45,남,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,[10],[0.9908612966537476],네.,[10],[0 0 0 0 0 0 0 0 0 0 1],['비인지왜곡'],D075,45,남,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,[10],[0.9982078075408936],반갑습니다.,[10],[0 0 0 0 0 0 0 0 0 0 1],['비인지왜곡'],D075,45,남,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,[2],[0.9671046733856201],시간대랑 그것 외에는.,[2],[0 0 1 0 0 0 0 0 0 0 0],['정신적 여과'],D075,45,남,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,[10],[0.9908612966537476],네.,[10],[0 0 0 0 0 0 0 0 0 0 1],['비인지왜곡'],D075,45,남,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 감정 분석 (KOTE 기반 감정 score 추출)

In [10]:
# 라벨 정의
label2id = {
    "이분법적 사고": 0,
    "과잉일반화": 1,
    "정신적 여과": 2,
    "긍정 무시": 3,
    "성급한 결론": 4,
    "과장 및 축소": 5,
    "감정적 추론": 6,
    "당위적 사고": 7,
    "개인화": 8,
    "낙인찍기": 9,
    "비인지왜곡": 10
}
id2label = {v: k for k, v in label2id.items()}
num_labels = len(label2id)  # 11

In [ ]:
# ===============================
# KOTE 감정 분석 (배치 + 진행바 + 길이 안전)
# ===============================

from transformers import pipeline
from tqdm.auto import tqdm
import pandas as pd

classifier = pipeline(
    "text-classification",
    model="tobykim/koelectra-44emotions",
    tokenizer="tobykim/koelectra-44emotions",
    device=0,                         # GPU
    return_all_scores=True,
    function_to_apply="softmax",
    batch_size=16,
    truncation=True,                 
    max_length=512                    
)

texts = df["analysis_text"].fillna("").tolist()

results = []

for preds in tqdm(classifier(texts), total=len(texts)):
    top = max(preds, key=lambda x: x["score"])
    results.append({
        "emotion_top1": top["label"],
        "emotion_top1_score": float(top["score"])
    })

emotion_df = pd.DataFrame(results)
df = pd.concat([df.reset_index(drop=True), emotion_df], axis=1)

df[["analysis_text", "emotion_top1", "emotion_top1_score"]].head()


Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


  0%|          | 0/54793 [00:00<?, ?it/s]

,analysis_text,emotion_top1,emotion_top1_score
0,안녕하세요.,LABEL_24,0.292347
1,네.,LABEL_24,0.275943
2,반갑습니다.,LABEL_42,0.129774
3,시간대랑 그것 외에는.,LABEL_24,0.127003
4,네.,LABEL_24,0.275943


In [18]:
save_path = "/content/drive/MyDrive/자연어처리_시험대체/output/emotion_df.csv"

df.to_csv(
    save_path,
    index=False,        # 인덱스 컬럼 저장 안 함
    encoding="utf-8-sig"  # 한글 안 깨지게 (중요)
)

print(f"저장 완료: {save_path}")


저장 완료: /content/drive/MyDrive/자연어처리_시험대체/output/emotion_df.csv


In [19]:
df.head()

,utterance_id,sentence_labels,sentence_scores,model_input_text,utterance_labels,utterance_multihot,utterance_label_names,id,age,gender,...,intensifiers_ratio_eojeol,intensifiers_ratio_morphs,single_cause_blame_count,single_cause_blame_ratio_eojeol,single_cause_blame_ratio_morphs,demandingness_count,demandingness_ratio_eojeol,demandingness_ratio_morphs,emotion_top1,emotion_top1_score
0,0,[10],[0.9981536269187927],안녕하세요.,[10],[0 0 0 0 0 0 0 0 0 0 1],['비인지왜곡'],D075,45,남,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LABEL_24,0.292347
1,1,[10],[0.9908612966537476],네.,[10],[0 0 0 0 0 0 0 0 0 0 1],['비인지왜곡'],D075,45,남,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LABEL_24,0.275943
2,2,[10],[0.9982078075408936],반갑습니다.,[10],[0 0 0 0 0 0 0 0 0 0 1],['비인지왜곡'],D075,45,남,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LABEL_42,0.129774
3,3,[2],[0.9671046733856201],시간대랑 그것 외에는.,[2],[0 0 1 0 0 0 0 0 0 0 0],['정신적 여과'],D075,45,남,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LABEL_24,0.127003
4,4,[10],[0.9908612966537476],네.,[10],[0 0 0 0 0 0 0 0 0 0 1],['비인지왜곡'],D075,45,남,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LABEL_24,0.275943
